# Regression Model B: Field 1

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 1 (F1)

Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E1" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E1" ),]
dE4=d4[which(d4$Estate=="E1" ),]
dE5=d5[which(d5$Estate=="E1" ),]

## Variables

Declare the variables. The variables in which pruning time, pruning cost, manuring time, manusring cost, P&D time and P&D cost contains zero values. In this case, we assume that the data is not available (NA). Then, we construct a new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL

foliar=d2$Diff

prun.time=dE3$Frequency
prun.time[prun.time == 0] <- NA

prun.cost=dE3$PruningCost
prun.cost[prun.cost == 0] <- NA

manuring.time=dE4$Frequency
manuring.time[manuring.time == 0] <- NA

manuring.cost=dE4$ManuringCost
manuring.cost[manuring.cost == 0] <- NA

pd.time=dE5$Frequency
pd.time[pd.time == 0] <- NA

pd.cost=dE5$PDCost
pd.cost[pd.cost == 0] <- NA


df1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Identify missing values in each variables

In this case, we impute the missing values. Imputation is the process of replacing missing data with substituted values.  

This study use MICE (Multivariate Imputation via Chained Equations). MICE (Multivariate Imputation via Chained Equations) is one of the commonly used package by R users. Creating multiple imputations as compared to a single imputation (such as mean) takes care of uncertainty in missing values.

MICE assumes that the missing data are Missing at Random (MAR), which means that the probability that a value is missing depends only on observed value and can be predicted using them. It imputes data on a variable by variable basis by specifying an imputation model per variable.


The mice package has function known as md.pattern(). It returns a tabular for of missing value present in each variable in a data set.

In [6]:
md.pattern(df1)

,year,month,yield,rainfall,foliar,pd.time,pd.cost,prun.time,prun.cost,manuring.time,manuring.cost,
11,1,1,1,1,1,1,1,1,1,1,1,0
8,1,1,1,1,1,1,1,0,0,1,1,2
8,1,1,1,1,1,1,1,1,1,0,0,2
9,1,1,1,1,1,0,0,1,1,1,1,2
4,1,1,1,1,1,1,1,0,0,0,0,4
2,1,1,1,1,1,0,0,0,0,1,1,4
2,1,1,1,1,1,0,0,1,1,0,0,4
,0,0,0,0,0,13,13,14,14,14,14,82


## Imputation

Here is an explanation of the parameters used:  
m  – Refers to 5 imputed data sets.  
maxit – Refers to no. of iterations taken to impute missing values.  
method – Refers to method used in imputation. We used predictive mean matching.

In [7]:
imputed_Data <- mice(df1[,-1:-4], m=5, maxit = 50, method = 'pmm', seed = 500)
summary(imputed_Data)


 iter imp variable
  1   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   3  prun.time  prun.cost  manuring.time

Multiply imputed data set
Call:
mice(data = df1[, -1:-4], m = 5, method = "pmm", maxit = 50, 
    seed = 500)
Number of multiple imputations:  5
Missing cells per column:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
            0            14            14            14            14 
      pd.time       pd.cost 
           13            13 
Imputation methods:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
        "pmm"         "pmm"         "pmm"         "pmm"         "pmm" 
      pd.time       pd.cost 
        "pmm"         "pmm" 
VisitSequence:
    prun.time     prun.cost manuring.time manuring.cost       pd.time 
            2             3             4             5             6 
      pd.cost 
            7 
PredictorMatrix:
              foliar prun.time prun.cost manuring.time manuring.cost pd.time
foliar             0         0         0             0             0       0
prun.time          1         0         1           

Since there are 5 imputed data sets, we select any using complete() function.
Get complete data (2nd out of 5)

Get the complete data after impute the missing value.

In [8]:
completeData <- complete(imputed_Data,2)
df2=data.frame(df1[,1:4],completeData)

## Correlation

Check the correlations with all variables. 

In [10]:
r1=cor(df2[,-1:-2])

corrplot(r1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(r1, type="upper")

## Train and test data

Split the dataset into train and test sets. After a model has been processed by using the training set, we test the model by making predictions against the test set. 

Because the data in the testing set already contains known values for the attribute that we want to predict, it is easy to determine whether the model's guesses are correct.

In this case, the datasets from May 2014 until December 2016 is considered for training dataset. The test dataset is from January 2017 until December 2017. 


In [11]:
train = df2[df2$year<=2016,]
test = df2[df2$year==2017,]
dim(train)
dim(test)

[1] 32 11

[1] 12 11

## Regression

Fit the data using multiple regression for all variables. 

In [12]:
k1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=train)
summary(k1)
plot(k1,which=1)
plot(k1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-73.710 -21.444  -1.998  30.242  73.213 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)   
(Intercept)   139.254086  45.934401   3.032  0.00593 **
rainfall        0.276905   0.112642   2.458  0.02191 * 
foliar         -2.330569   3.137863  -0.743  0.46517   
prun.time      -2.626169   9.948102  -0.264  0.79414   
prun.cost       0.002864   0.002634   1.088  0.28806   
manuring.time -47.485898  35.976019  -1.320  0.19984   
manuring.cost  -0.001122   0.001027  -1.092  0.28597   
pd.time        19.961931  15.664532   1.274  0.21527   
pd.cost         0.002596   0.007718   0.336  0.73964   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 44.95 on 23 degrees of freedom
Multiple R-squared:  0.3321,	Adjusted R-squared:  0.09977 
F-stat

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 

The stepwise method incorporate with VIF factor is better than anova method in order to select the significant variables. 

### Using ANOVA

In [13]:
anova(k1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,12449.2113,12449.2113,6.16124317,0.02079428
foliar,1,153.6359,153.6359,0.07603599,0.78520533
prun.time,1,1011.6385,1011.6385,0.50067032,0.48631448
prun.cost,1,3718.5777,3718.5777,1.84036248,0.18807594
manuring.time,1,323.8890,323.8890,0.16029603,0.69257869
manuring.cost,1,1088.5764,1088.5764,0.53874769,0.47036912
pd.time,1,4132.6507,4132.6507,2.04529146,0.16612297
pd.cost,1,228.6266,228.6266,0.11314964,0.73963613
Residuals,23,46473.0660,2020.5681,NA,NA


### Using stepwise

In [14]:
selectedMod <- step(k1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=250.99
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- prun.time      1     140.8 46614 249.09
- pd.cost        1     228.6 46702 249.15
- foliar         1    1114.6 47588 249.75
- prun.cost      1    2389.9 48863 250.59
- manuring.cost  1    2411.2 48884 250.61
<none>                       46473 250.99
- pd.time        1    3281.3 49754 251.17
- manuring.time  1    3520.3 49993 251.32
- rainfall       1   12210.5 58684 256.45

Step:  AIC=249.09
yield ~ rainfall + foliar + prun.cost + manuring.time + manuring.cost + 
    pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- pd.cost        1     291.5 46905 247.28
- foliar         1    1256.1 47870 247.94
- prun.cost      1    2269.6 48883 248.61
- manuring.cost  1    2297.8 48912 248.62
<none>                       46614 249.09
- pd.time        1    3264.5 49878 249.25
- manuring.time  1    3491.7 50106 249.40
- 


Call:
lm(formula = yield ~ rainfall + prun.cost, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-70.442 -33.769  -0.574  34.662  90.867 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1.061e+02  1.818e+01   5.834 2.51e-06 ***
rainfall    2.011e-01  7.881e-02   2.552   0.0162 *  
prun.cost   3.603e-03  2.368e-03   1.522   0.1389    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 42.71 on 29 degrees of freedom
Multiple R-squared:  0.2396,	Adjusted R-squared:  0.1872 
F-statistic:  4.57 on 2 and 29 DF,  p-value: 0.01883


 rainfall prun.cost 
 1.001467  1.001467 



Call:
lm(formula = yield ~ rainfall + prun.cost, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-70.442 -33.769  -0.574  34.662  90.867 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1.061e+02  1.818e+01   5.834 2.51e-06 ***
rainfall    2.011e-01  7.881e-02   2.552   0.0162 *  
prun.cost   3.603e-03  2.368e-03   1.522   0.1389    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 42.71 on 29 degrees of freedom
Multiple R-squared:  0.2396,	Adjusted R-squared:  0.1872 
F-statistic:  4.57 on 2 and 29 DF,  p-value: 0.01883


## Prediction

The performance of the model is evaluated by comparing the estimated value with the actual value. In this case, the test dataset (2017 datasets) is used to examine the performance of model with significant variables. 

Construct the table and graph that represents the estimated and observed value. 

In [15]:
estimate=predict(selectedMod,newdata=test)
compare=data.frame(test$yield,estimate)
names(compare)=c("Actual","Estimate")
compare

plot(compare$Actual,type="b",col="blue",xlab="Actual",ylab="Predicted",pch=19)
lines(fitted(selectedMod),col="red",type="b",pch=18)
legend("topleft",legend=c("Actual","Predicted"),col=c("blue","red"),lty=1,cex=0.8)

,Actual,Estimate
,<dbl>,<dbl>
33,186.702,181.6019
34,108.890,122.0093
35,153.995,154.4923
36,108.133,171.3830
37,153.934,165.5500
38,72.808,145.7804
39,150.154,127.7549
40,178.923,146.5850
41,175.160,162.1555


## Root mean square error (RMSE)

Calculate the root mean square error (RMSE) of the model. Extract the coefficient of determination (  ) to know how close the data to the fitted regression line. 

In [16]:
rmse=rmse(test$yield,estimate )
fit=c(RMSE = rmse, R2=summary(k1)$r.squared)
fit

RMSE         R2 
65.0530990  0.3320904